# Assigment 6.12: Classiq
$\textit{Adriano Lusso}$

## Imports

In [6]:
from classiq import *

## Auxiliary functions

In [15]:
@qfunc
def diffuser_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)

@qfunc
def zero_diffuser(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),action=lambda: diffuser_oracle)


def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    probs = [0 for _ in range(n_vertices)]
    
    probs[(i+1)% 16]=0.5
    probs[(i-1)% 16]=0.5
    
    if i ==0:
      probs[15]=0
      probs[1]=1
    elif i==15:
      probs[0]=0
      probs[14]=1
        
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=probs, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuser(adjacent_vertices)))

@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)


@qfunc
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

@qfunc
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
      W_iteration(i,vertices,adjacent_vertices)


@qfunc
def edge_oracle(output:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  output |= (((vertices+adjacent_vertices)%2) ==1)

@qfunc
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,iteration= lambda i: SWAP(x[i],y[i]))

## Main function and hyperparameters

In [17]:
# hyperparameters
n_vertices = 16
size = 4

@qfunc
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)


## Run the code 

In [11]:
qmod = create_model(main)
program = synthesize(qmod)
show(program)

Opening: https://platform.classiq.io/circuit/4650928d-1923-42e7-a800-9d8e43de19e5?version=0.43.3


## Resulting circuit

![](./circuit.jpg)

![](./circuit2.jpg)